<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

#  Multi-Query-Retriever

This method can automatically use a ChatModel to make slight variations of your initial query to help attempt overcome any issues with cosine similarity distances, this allows you to phrase things more like a general query question rather than a pure document similarity look-up.

In [1]:
# Build a sample vectorDB
from langchain.vectorstores import Chroma
from langchain.document_loaders import WikipediaLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

### Loader

In [2]:
loader = WikipediaLoader(query='MKUltra')
documents = loader.load()

/Users/koichirosuzuki/.pyenv/versions/3.11.0/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/koichirosuzuki/.pyenv/versions/3.11.0/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [3]:
len(documents)

24

### Split Documents

In [4]:
# split it into chunks
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
docs = text_splitter.split_documents(documents)

Created a chunk of size 528, which is longer than the specified 500
Created a chunk of size 504, which is longer than the specified 500
Created a chunk of size 772, which is longer than the specified 500


### OpenAI Connection for Embeddings

In [5]:
import os
f = open('/Users/koichirosuzuki/Desktop/desktop_openai.txt')
os.environ['OPENAI_API_KEY'] = 'yourapikey'
api_key = f.read()
embedding_function = OpenAIEmbeddings()

/Users/koichirosuzuki/.pyenv/versions/3.11.0/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [6]:
# docs

### Embed Documents for ChromaDB

In [7]:
# load it into Chroma
db = Chroma.from_documents(docs, embedding_function,persist_directory='./mk_ultra')
db.persist()

### Use Chat Model to Multi Query

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
question="When was this declassified?"
llm = ChatOpenAI(temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(retriever=db.as_retriever(),llm=llm)

/Users/koichirosuzuki/.pyenv/versions/3.11.0/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [13]:
# Set logging for the queries
import logging
logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

In [14]:
unique_docs = retriever_from_llm.get_relevant_documents(query=question)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What is the date of the declassification of this information?', '2. Can you provide the specific time when this was declassified?', '3. Do you know the exact moment when this information became declassified?']


In [15]:
len(unique_docs)

6

In [16]:
print(unique_docs[0].page_content)

== See also ==
CIA activities in the United States
Frank Olson
MKUltra
MKNAOMI
MKOFTEN
Project CHATTER
Unethical human experimentation in the United States


== Footnotes ==


== References ==
Maret, Susan (2018-02-16). "Murky Projects and Uneven Information Policies: A Case Study of the Psychological Strategy Board and CIA". Secrecy and Society. 1 (2). doi:10.31979/2377-6188.2018.010206. ISSN 2377-6188.
Ronson, Jon (2004). The Men Who Stare at Goats. London: Picador. ISBN 0-330-37547-4. OCLC 56653467.


== External links ==
Transcriptions of Declassified Project Artichoke Documents at the Wayback Machine (archived May 1, 2011)
